In [2]:
!pip install ultralytics

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/dc/cf/978e2d8ffcd4b22793d4857f965fdac90514616ac9a91a4c5a29eaf9f586/ultralytics-8.1.42-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/40.4 kB ? eta -:--:--
     ------------------------------ --------- 30.7/40.4 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 40.4/40.4 kB 479.0 kB/s eta 0:00:00
  Obtaining dependency information for torchvision>=0.9.0 from https://files.pythonhosted.org/packages/c6/75/d869f600fc33df8b8ca99943e165a4ca23b73c68dc1942098fde0a6b46f3/torchvision-0.17.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for thop>=0.1.1 from https://files.pythonhosted.org/packages/bb/0f/72beeab4ff5221dc47127c80f8834b4bcd0cb36f6ba91c0b1d04a1233403/thop-0.1.1.post2209072238-py3-none-any.whl.metadata
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl.metadata (2.7 kB)
  Obtaining dependency information for torch>

   ----------- --------------------------- 58.2/198.6 MB 377.1 kB/s eta 0:06:13
   ----------- --------------------------- 58.2/198.6 MB 377.6 kB/s eta 0:06:12
   ----------- --------------------------- 58.2/198.6 MB 377.6 kB/s eta 0:06:12
   ----------- --------------------------- 58.3/198.6 MB 378.4 kB/s eta 0:06:11
   ----------- --------------------------- 58.3/198.6 MB 377.6 kB/s eta 0:06:12
   ----------- --------------------------- 58.3/198.6 MB 378.4 kB/s eta 0:06:11
   ----------- --------------------------- 58.4/198.6 MB 378.7 kB/s eta 0:06:11
   ----------- --------------------------- 58.4/198.6 MB 378.9 kB/s eta 0:06:10
   ----------- --------------------------- 58.4/198.6 MB 378.7 kB/s eta 0:06:11
   ----------- --------------------------- 58.5/198.6 MB 378.9 kB/s eta 0:06:10
   ----------- --------------------------- 58.5/198.6 MB 379.3 kB/s eta 0:06:10
   ----------- --------------------------- 58.5/198.6 MB 378.9 kB/s eta 0:06:10
   ----------- -------------------------

# IMPORTING LIBARIES

In [3]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*

C:\Users\lenovo1\anaconda3\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
model=YOLO('yolov9c.pt')

class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

100%|█████████████████████████████████████████████████████████████████████████████| 49.4M/49.4M [00:30<00:00, 1.73MB/s]


In [5]:
tracker=Tracker()
count=0

In [ ]:
cap=cv2.VideoCapture('traffictrim.mp4')

down={}
counter_down=set()
while True:    
    ret,frame = cap.read()
    if not ret:
        break
    count += 1

    results=model.predict(frame)

    a=results[0].boxes.data
    a = a.detach().cpu().numpy()  
    px=pd.DataFrame(a).astype("float")
    #print(px)
    
    list=[]
             
    for index,row in px.iterrows():
#        print(row) 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'car' in c:
            list.append([x1,y1,x2,y2])


    bbox_id=tracker.update(list)
    #print(bbox_id)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2
        # cv2.circle(frame,(cx,cy),4,(0,0,255),-1) #draw ceter points of bounding box
        # cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
        # cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)



        y = 308
        offset = 7
    
        ''' condition for red line '''
        if y < (cy + offset) and y > (cy - offset):
          ''' this if condition is putting the id and the circle on the object when the center of the object touched the red line.'''
          
          down[id]=cy   #cy is current position. saving the ids of the cars which are touching the red line first. 
          #This will tell us the travelling direction of the car.
          if id in down:         
           cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
           cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
           counter_down.add(id) 

    # # line
    text_color = (255,255,255)  # white color for text
    red_color = (0, 0, 255)  # (B, G, R)   
    
    # print(down)
    cv2.line(frame,(282,308),(1004,308),red_color,3)  #  starting cordinates and end of line cordinates
    cv2.putText(frame,('red line'),(280,308),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA) 


    downwards = (len(counter_down))
    cv2.putText(frame,('going down - ')+ str(downwards),(60,40),cv2.FONT_HERSHEY_SIMPLEX, 0.5, red_color, 1, cv2.LINE_AA) 
    


    # cv2.line(frame,(282,308),(1004,308),red_color,3)  #  starting cordinates and end of line cordinates
    # cv2.putText(frame,('red line'),(280,308),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)    
    
    cv2.imshow("frames", frame)
    if cv2.waitKey(1)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()


0: 384x640 1 person, 4 cars, 2492.9ms
Speed: 29.9ms preprocess, 2492.9ms inference, 24.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 4 cars, 1446.8ms
Speed: 11.3ms preprocess, 1446.8ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 4 cars, 1506.0ms
Speed: 6.0ms preprocess, 1506.0ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 4 cars, 1 traffic light, 1471.1ms
Speed: 10.0ms preprocess, 1471.1ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 4 cars, 1351.4ms
Speed: 5.0ms preprocess, 1351.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 4 cars, 1 traffic light, 1360.4ms
Speed: 4.0ms preprocess, 1360.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 4 cars, 1457.1ms
Speed: 3.0ms preprocess, 1457.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384,


0: 384x640 1 person, 5 cars, 1 traffic light, 1404.2ms
Speed: 4.0ms preprocess, 1404.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 5 cars, 1972.7ms
Speed: 4.0ms preprocess, 1972.7ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 traffic light, 2027.5ms
Speed: 5.0ms preprocess, 2027.5ms inference, 6.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 traffic light, 2190.1ms
Speed: 5.0ms preprocess, 2190.1ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 5 cars, 1 traffic light, 2138.3ms
Speed: 7.0ms preprocess, 2138.3ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 5 cars, 2244.0ms
Speed: 5.0ms preprocess, 2244.0ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 2062.5ms
Speed: 5.0ms preprocess, 2062.5ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 6


0: 384x640 4 cars, 1 traffic light, 1997.3ms
Speed: 4.7ms preprocess, 1997.3ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 2045.3ms
Speed: 4.2ms preprocess, 2045.3ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 2141.8ms
Speed: 5.0ms preprocess, 2141.8ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 traffic light, 2179.9ms
Speed: 4.0ms preprocess, 2179.9ms inference, 6.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 traffic light, 2159.9ms
Speed: 5.0ms preprocess, 2159.9ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 2096.1ms
Speed: 5.1ms preprocess, 2096.1ms inference, 5.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 2206.4ms
Speed: 6.0ms preprocess, 2206.4ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 traffic light, 2173.5ms
Speed: 


0: 384x640 1 person, 3 cars, 1 traffic light, 1993.1ms
Speed: 4.7ms preprocess, 1993.1ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 cars, 1 traffic light, 2014.8ms
Speed: 4.0ms preprocess, 2014.8ms inference, 5.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 cars, 1 traffic light, 2332.3ms
Speed: 4.9ms preprocess, 2332.3ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 cars, 1 traffic light, 2029.5ms
Speed: 5.0ms preprocess, 2029.5ms inference, 5.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 cars, 1 traffic light, 2124.6ms
Speed: 6.0ms preprocess, 2124.6ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 cars, 1 traffic light, 2143.1ms
Speed: 5.1ms preprocess, 2143.1ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 cars, 1 traffic light, 2046.5ms
Speed: 7.0ms p


0: 384x640 3 persons, 4 cars, 2132.8ms
Speed: 4.0ms preprocess, 2132.8ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 4 cars, 1941.1ms
Speed: 5.4ms preprocess, 1941.1ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 4 cars, 2091.4ms
Speed: 4.2ms preprocess, 2091.4ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 5 cars, 2676.0ms
Speed: 7.2ms preprocess, 2676.0ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 5 cars, 2814.8ms
Speed: 8.3ms preprocess, 2814.8ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 6 cars, 2208.6ms
Speed: 5.1ms preprocess, 2208.6ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 5 cars, 2552.1ms
Speed: 6.0ms preprocess, 2552.1ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 5 ca

...

Speed: 5.0ms preprocess, 1359.4ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 traffic light, 1374.3ms
Speed: 4.0ms preprocess, 1374.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 1 traffic light, 1355.3ms
Speed: 4.0ms preprocess, 1355.3ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 traffic light, 1302.5ms
Speed: 5.0ms preprocess, 1302.5ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 1 traffic light, 1351.4ms
Speed: 5.0ms preprocess, 1351.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 traffic light, 1554.8ms
Speed: 4.0ms preprocess, 1554.8ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 1 traffic light, 1500.0ms
Speed: 4.0ms preprocess, 1500.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)




0: 384x640 1 person, 7 cars, 1 truck, 1660.6ms
Speed: 5.0ms preprocess, 1660.6ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 7 cars, 1 truck, 1485.0ms
Speed: 3.0ms preprocess, 1485.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 1 truck, 1704.4ms
Speed: 5.0ms preprocess, 1704.4ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 1 truck, 1854.0ms
Speed: 6.0ms preprocess, 1854.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 2026.6ms
Speed: 4.0ms preprocess, 2026.6ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 1537.9ms
Speed: 4.0ms preprocess, 1537.9ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 1 truck, 1 traffic light, 1413.2ms
Speed: 4.0ms preprocess, 1413.2ms inference, 3.0ms postprocess per image

Speed: 4.0ms preprocess, 1526.9ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 6 cars, 1 truck, 1304.5ms
Speed: 4.0ms preprocess, 1304.5ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 6 cars, 1 truck, 1 traffic light, 1311.5ms
Speed: 5.0ms preprocess, 1311.5ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 truck, 1 traffic light, 1316.4ms
Speed: 4.0ms preprocess, 1316.4ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 truck, 1 traffic light, 1231.7ms
Speed: 4.0ms preprocess, 1231.7ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 truck, 1 traffic light, 2056.5ms
Speed: 4.0ms preprocess, 2056.5ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 6 cars, 1 truck, 1847.1ms
Speed: 6.0ms preprocess, 1847.1ms inference, 2.0ms postprocess per image at shap


0: 384x640 1 person, 7 cars, 2 trucks, 1 traffic light, 2820.6ms
Speed: 17.0ms preprocess, 2820.6ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 2 trucks, 1 traffic light, 1349.6ms
Speed: 22.9ms preprocess, 1349.6ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 1 motorcycle, 1 truck, 1 traffic light, 1309.5ms
Speed: 4.0ms preprocess, 1309.5ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 1 motorcycle, 1 truck, 1 traffic light, 1617.6ms
Speed: 5.0ms preprocess, 1617.6ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 7 cars, 1 motorcycle, 1 truck, 1 traffic light, 1805.3ms
Speed: 6.0ms preprocess, 1805.3ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 1 motorcycle, 2 trucks, 1 traffic light, 1765.7ms
Speed: 8.0ms preprocess, 1765.7ms inference, 4.0ms po


0: 384x640 1 person, 7 cars, 1 motorcycle, 1 truck, 1357.3ms
Speed: 4.0ms preprocess, 1357.3ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 8 cars, 1 motorcycle, 1 truck, 1 traffic light, 1561.8ms
Speed: 4.0ms preprocess, 1561.8ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 8 cars, 1 motorcycle, 1 truck, 1 traffic light, 1578.2ms
Speed: 7.0ms preprocess, 1578.2ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 7 cars, 1 motorcycle, 1 truck, 1 traffic light, 1554.1ms
Speed: 6.0ms preprocess, 1554.1ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 7 cars, 1 motorcycle, 1 truck, 1 traffic light, 2018.3ms
Speed: 15.0ms preprocess, 2018.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 7 cars, 1 motorcycle, 1 truck, 1 traffic light, 1392.7ms
Speed: 4.0ms preprocess, 1392.7ms inference,